In [15]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

len4 blp impute initial lag value

In [16]:
data=pd.read_csv('../merged/len4_ndb_agg_blp_DropNever.csv',encoding="shift-jis",index_col=0)
data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_3999.0,id_4413.0,id_4490.0,id_6250.0,id_6290.0,id_7990.0,q_share,r_share,generic_share_q,generic_share_r
0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,2.489539e+07,2014,1,...,0,0,0,0,0,0,0.150667,0.074626,0.076961,0.059379
1,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,1.024590e+07,2014,1,...,0,0,0,0,0,0,0.062008,0.054392,0.076961,0.059379
2,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,5.025912e+07,2014,0,...,0,0,0,0,0,0,0.304168,0.150655,0.076961,0.059379
3,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,2.742935e+07,2014,0,...,0,0,0,0,0,0,0.166002,0.145613,0.076961,0.059379
4,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,63.1,0.0,5.339043e+06,2014,1,...,0,0,0,0,0,0,0.032312,0.077089,0.076961,0.059379


In [17]:
data.columns[71:103]

Index(['id_1147.0', 'id_1149.0', 'id_1219.0', 'id_1231.0', 'id_1319.0',
       'id_1324.0', 'id_1329.0', 'id_2189.0', 'id_2239.0', 'id_2259.0',
       'id_2290.0', 'id_2319.0', 'id_2325.0', 'id_2329.0', 'id_2399.0',
       'id_2454.0', 'id_2459.0', 'id_2529.0', 'id_2590.0', 'id_2646.0',
       'id_2649.0', 'id_2655.0', 'id_2659.0', 'id_3327.0', 'id_3399.0',
       'id_3992.0', 'id_3999.0', 'id_4413.0', 'id_4490.0', 'id_6250.0',
       'id_6290.0', 'id_7990.0'],
      dtype='object')

In [18]:
for target in ["lag_mean_price","lag_sum_quantity"]:
    data_est=data.dropna(subset=[target])
    ids=data_est.iloc[:,71:103]
    X=data_est[["year"]+list(ids.columns)]
    y=data_est[target]
    # X = sm.add_constant(X)  # 定数項を追加
    model = sm.OLS(y, X).fit()

    # モデルの要約を表示
    print(model.summary())
    missing=data[data[target].isnull()]
    missing=missing[["year"]+list(ids.columns)]
    # missing["const"]=1
    predicted_values=model.predict(missing)
    # predicted_values
    missing[target]=predicted_values
    data.update(missing)

                            OLS Regression Results                            
Dep. Variable:         lag_mean_price   R-squared:                       0.920
Model:                            OLS   Adj. R-squared:                  0.920
Method:                 Least Squares   F-statistic:                     8824.
Date:                Sat, 09 Sep 2023   Prob (F-statistic):               0.00
Time:                        21:54:32   Log-Likelihood:            -1.5474e+05
No. Observations:               24448   AIC:                         3.096e+05
Df Residuals:                   24415   BIC:                         3.098e+05
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
year          -4.3043      0.434     -9.927      0.0

In [19]:
# data.columns
data.to_csv('../merged/len4_ndb_agg_blp_imputed_lags.csv',encoding="shift-jis")

In [20]:
data.shape

(25798, 107)